In [1]:
import requests
import json
import pandas as pd

# Your API key (REGENERATE THIS AFTER USING!)
API_KEY = 'bkRO9B7Y_6B31H22F7iofhMXCaOK0K6I9BN6U9y52BVXCxEtU_fMoNS6Tpw_w903tIIzmoXJ9mdVMMw_laU8FiLN6NQv4GFpi0e2SM-j00G50uigrOpm7R6EBsuTaXYx'

def search_yelp(term, location, categories='cafes,desserts', limit=50):
    """Search Yelp for businesses matching keywords"""
    
    headers = {'Authorization': f'Bearer {API_KEY}'}
    
    params = {
        'term': term,
        'location': location,
        'categories': categories,
        'limit': limit
    }
    
    url = 'https://api.yelp.com/v3/businesses/search'
    response = requests.get(url, headers=headers, params=params)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error {response.status_code}: {response.text}")
        return None

def get_business_reviews(business_id):
    """Get up to 3 reviews for a business"""
    
    headers = {'Authorization': f'Bearer {API_KEY}'}
    url = f'https://api.yelp.com/v3/businesses/{business_id}/reviews'
    
    response = requests.get(url, headers=headers, params={'limit': 3})
    
    if response.status_code == 200:
        return response.json()
    else:
        return None

# Search for black sesame in major cities
cities = [
    'San Francisco, CA',
    'Los Angeles, CA', 
    'New York, NY',
    'Seattle, WA',
    'Portland, OR',
    'Chicago, IL',
    'Austin, TX',
    'Boston, MA'
]

print("Searching for 'black sesame' mentions...")
all_businesses = []

for city in cities:
    print(f"\n🔍 Searching in {city}...")
    results = search_yelp('black sesame', city)
    
    if results and 'businesses' in results:
        businesses = results['businesses']
        print(f"   Found {len(businesses)} businesses")
        
        for biz in businesses:
            business_data = {
                'name': biz['name'],
                'city': biz['location']['city'],
                'state': biz['location']['state'],
                'address': biz['location']['address1'],
                'zip_code': biz['location']['zip_code'],
                'latitude': biz['coordinates']['latitude'],
                'longitude': biz['coordinates']['longitude'],
                'rating': biz['rating'],
                'review_count': biz['review_count'],
                'categories': ', '.join([c['title'] for c in biz.get('categories', [])]),
                'price': biz.get('price', 'N/A'),
                'business_id': biz['id']
            }
            
            all_businesses.append(business_data)
            print(f"   ✓ {biz['name']}")

# Save to CSV
df = pd.DataFrame(all_businesses)
df.to_csv('black_sesame_businesses_2025.csv', index=False)

print(f"\n✅ Total: {len(all_businesses)} businesses found")
print(f"📊 Data saved to: black_sesame_businesses_2025.csv")

# Summary statistics
print("\n📈 Summary:")
print(f"Cities covered: {df['city'].nunique()}")
print(f"Average rating: {df['rating'].mean():.2f}")
print(f"Total reviews: {df['review_count'].sum()}")

print("\nTop 5 cities:")
print(df['city'].value_counts().head())

Searching for 'black sesame' mentions...

🔍 Searching in San Francisco, CA...
   Found 50 businesses
   ✓ christine's
   ✓ Powder
   ✓ Breadbelly
   ✓ Kiss Of Matcha
   ✓ Stonemill Matcha
   ✓ OH Dessert Cafe
   ✓ Little Swan Bakery Cafe
   ✓ Maruwu Seicha
   ✓ Uji Time Dessert
   ✓ Tokyo Cream
   ✓ U :Dessert Story
   ✓ Sweet Glory
   ✓ Kowloon Tong Dessert Cafe
   ✓ Buoy
   ✓ Golden Island Cafe
   ✓ Matcha Cafe Maiko
   ✓ A Buttercup Dessert House
   ✓ Golden Morsels
   ✓ Garden Creamery
   ✓ Metro Hong Kong Dessert
   ✓ The French Spot
   ✓ Urban Ritual
   ✓ Sweethoney Dessert
   ✓ K Minamoto
   ✓ Jinshan Cafe
   ✓ Marco Polo Italian Ice Cream
   ✓ The Game Parlour
   ✓ Aqua Club Dessert & Beverage
   ✓ Polly Ann Ice Cream
   ✓ Telescope Coffee
   ✓ Jane On Third
   ✓ ZERO &
   ✓ Do-Re-Mi
   ✓ Hila Gelato Caffe
   ✓ Kiss of Matcha
   ✓ Tadaima - Mission
   ✓ Sweet Mango Dessert & Cafe
   ✓ NaYa Dessert Cafe
   ✓ Bake Sum
   ✓ Uji Time Dessert 
   ✓ Milkbomb Ice Cream
   ✓ Sixth Cour

In [ ]:
import requests
import pandas as pd
import time

API_KEY = 'bkRO9B7Y_6B31H22F7iofhMXCaOK0K6I9BN6U9y52BVXCxEtU_fMoNS6Tpw_w903tIIzmoXJ9mdVMMw_laU8FiLN6NQv4GFpi0e2SM-j00G50uigrOpm7R6EBsuTaXYx'
HEADERS = {'Authorization': f'Bearer {API_KEY}'}

# First: debug the reviews endpoint to see what we're actually getting back
print("🔍 Debugging the reviews endpoint...\n")

# Pick a known popular business
test_url = 'https://api.yelp.com/v3/businesses/search'
test_resp = requests.get(test_url, headers=HEADERS,
                         params={'term': 'black sesame', 'location': 'San Francisco', 'limit': 1})
test_biz = test_resp.json()['businesses'][0]
test_id = test_biz['id']
print(f"Test business: {test_biz['name']} (id: {test_id})")

# Try reviews endpoint
rev_url = f'https://api.yelp.com/v3/businesses/{test_id}/reviews'
rev_resp = requests.get(rev_url, headers=HEADERS, params={'limit': 20, 'sort_by': 'newest'})
print(f"Reviews endpoint status: {rev_resp.status_code}")
print(f"Reviews response keys: {list(rev_resp.json().keys())}")
print(f"Full response (first 500 chars): {rev_resp.text[:500]}")
print(f"Number of reviews returned: {len(rev_resp.json().get('reviews', []))}")

# Also try without sort_by param in case that's the issue
rev_resp2 = requests.get(rev_url, headers=HEADERS, params={'limit': 3})
print(f"\nWithout sort_by - status: {rev_resp2.status_code}")
print(f"Reviews returned: {len(rev_resp2.json().get('reviews', []))}")
print(f"Response: {rev_resp2.text[:500]}")

In [4]:
import json
import pandas as pd

print("Loading Yelp Open Dataset...")

# 1. Load reviews
print("Loading reviews...")
reviews = []
with open('yelp_academic_dataset_review.json', 'r', encoding='utf-8') as f:
    for i, line in enumerate(f):
        if i % 500000 == 0:
            print(f"  Processed {i:,} reviews...")
        reviews.append(json.loads(line))

df_reviews = pd.DataFrame(reviews)
print(f"✅ Loaded {len(df_reviews):,} reviews")

# Check what columns we have
print(f"\nReview columns: {list(df_reviews.columns)}")

# 2. Load businesses
print("\nLoading businesses...")
businesses = []
with open('yelp_academic_dataset_business.json', 'r', encoding='utf-8') as f:
    for line in f:
        businesses.append(json.loads(line))

df_business = pd.DataFrame(businesses)
print(f"✅ Loaded {len(df_business):,} businesses")
print(f"Business columns: {list(df_business.columns)}")

# 3. Define search terms
flavors = {
    'black sesame': ['black sesame', 'kuro goma', 'heugimja'],
    'matcha': ['matcha', 'green tea latte', 'matcha latte'],
    'ube': ['ube', 'purple yam', 'ube latte']
}

# 4. Search for each flavor
print("\n" + "="*70)
print("SEARCHING FOR ASIAN FLAVORS IN REVIEWS")
print("="*70)

results = {}
for flavor_name, keywords in flavors.items():
    print(f"\n🔍 Searching for: {flavor_name}")
    
    pattern = '|'.join(keywords)
    matches = df_reviews[
        df_reviews['text'].str.contains(pattern, case=False, na=False)
    ]
    
    print(f"   Found {len(matches):,} reviews")
    results[flavor_name] = matches

# 5. Merge with business data
print("\n" + "="*70)
print("ANALYZING BY CITY AND BUSINESS TYPE")
print("="*70)

cafe_keywords = ['cafe', 'coffee', 'dessert', 'ice cream', 'bakery', 'tea', 'boba']
cafe_pattern = '|'.join(cafe_keywords)

detailed_results = {}

for flavor_name, flavor_reviews in results.items():
    print(f"\n📊 {flavor_name.upper()}")
    
    # Merge with business info - use correct column names
    merged = flavor_reviews.merge(
        df_business,
        on='business_id',
        how='left'
    )
    
    # Filter to cafes/dessert shops (check if categories column exists)
    if 'categories' in merged.columns:
        cafe_data = merged[
            merged['categories'].str.contains(cafe_pattern, case=False, na=False)
        ]
    else:
        cafe_data = merged
    
    print(f"   Total mentions: {len(merged):,}")
    print(f"   In cafes/desserts: {len(cafe_data):,}")
    print(f"   Unique businesses: {cafe_data['business_id'].nunique():,}")
    
    # Top cities
    if 'city' in cafe_data.columns:
        print(f"\n   Top 10 cities:")
        city_counts = cafe_data['city'].value_counts().head(10)
        for city, count in city_counts.items():
            print(f"      {city}: {count}")
    
    detailed_results[flavor_name] = cafe_data

# 6. Time trends
print("\n" + "="*70)
print("YEARLY TRENDS")
print("="*70)

for flavor_name, data in detailed_results.items():
    print(f"\n📈 {flavor_name.upper()}")
    data['year'] = pd.to_datetime(data['date']).dt.year
    yearly = data.groupby('year').size()
    
    for year, count in yearly.items():
        print(f"   {year}: {count:,} mentions")
    
    if len(yearly) > 1:
        first_year = yearly.iloc[0]
        last_year = yearly.iloc[-1]
        growth = ((last_year - first_year) / first_year * 100) if first_year > 0 else 0
        print(f"   📊 Growth: {growth:.1f}%")

# 7. Save results
print("\n" + "="*70)
print("SAVING RESULTS")
print("="*70)

for flavor_name, data in detailed_results.items():
    filename = f'{flavor_name.replace(" ", "_")}_analysis.csv'
    data.to_csv(filename, index=False)
    print(f"✅ Saved {filename} ({len(data):,} records)")

# 8. Create comparison summary - use correct column names
comparison_summary = []
for flavor_name, data in detailed_results.items():
    summary = {
        'Flavor': flavor_name,
        'Total Mentions': len(data),
        'Unique Businesses': data['business_id'].nunique(),
    }
    
    # Add optional columns if they exist
    if 'city' in data.columns:
        summary['Cities'] = data['city'].nunique()
    
    # Check for rating columns (could be 'stars', 'rating', or in review data)
    if 'stars_x' in data.columns:  # from review
        summary['Avg Review Rating'] = data['stars_x'].mean()
    elif 'stars_y' in data.columns:  # from business
        summary['Avg Business Rating'] = data['stars_y'].mean()
    elif 'stars' in data.columns:
        summary['Avg Rating'] = data['stars'].mean()
    
    if 'review_count' in data.columns:
        summary['Total Reviews'] = data['review_count'].sum()
    
    comparison_summary.append(summary)

df_summary = pd.DataFrame(comparison_summary)
df_summary = df_summary.round(2)
df_summary.to_csv('flavor_comparison_summary.csv', index=False)

print("\n" + "="*70)
print("FINAL COMPARISON")
print("="*70)
print(df_summary.to_string(index=False))

# 9. Prepare locations for Census matching
print("\n" + "="*70)
print("PREPARING CENSUS DATA MATCHING")
print("="*70)

black_sesame_data = detailed_results['black sesame']

# Get columns that exist
location_cols = ['business_id', 'name']
if 'city' in black_sesame_data.columns:
    location_cols.append('city')
if 'state' in black_sesame_data.columns:
    location_cols.append('state')
if 'latitude' in black_sesame_data.columns:
    location_cols.append('latitude')
if 'longitude' in black_sesame_data.columns:
    location_cols.append('longitude')
if 'postal_code' in black_sesame_data.columns:
    location_cols.append('postal_code')

black_sesame_locations = black_sesame_data[location_cols].drop_duplicates('business_id')

print(f"Found {len(black_sesame_locations)} unique black sesame businesses")
black_sesame_locations.to_csv('black_sesame_locations_for_census.csv', index=False)
print("✅ Saved black_sesame_locations_for_census.csv")

print("\n✅ Analysis complete!")

Loading Yelp Open Dataset...
Loading reviews...
  Processed 0 reviews...
  Processed 500,000 reviews...
  Processed 1,000,000 reviews...
  Processed 1,500,000 reviews...
  Processed 2,000,000 reviews...
  Processed 2,500,000 reviews...
  Processed 3,000,000 reviews...
  Processed 3,500,000 reviews...
  Processed 4,000,000 reviews...
  Processed 4,500,000 reviews...
  Processed 5,000,000 reviews...
  Processed 5,500,000 reviews...
  Processed 6,000,000 reviews...
  Processed 6,500,000 reviews...
✅ Loaded 6,990,280 reviews

Review columns: ['review_id', 'user_id', 'business_id', 'stars', 'useful', 'funny', 'cool', 'text', 'date']

Loading businesses...
✅ Loaded 150,346 businesses
Business columns: ['business_id', 'name', 'address', 'city', 'state', 'postal_code', 'latitude', 'longitude', 'stars', 'review_count', 'is_open', 'attributes', 'categories', 'hours']

SEARCHING FOR ASIAN FLAVORS IN REVIEWS

🔍 Searching for: black sesame
   Found 698 reviews

🔍 Searching for: matcha
   Found 6,06

/var/folders/j5/twx6v7jj4nq17ymrbt5c1klw0000gn/T/ipykernel_2982/3526023640.py:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['year'] = pd.to_datetime(data['date']).dt.year


✅ Saved matcha_analysis.csv (5,059 records)
✅ Saved ube_analysis.csv (10,162 records)

FINAL COMPARISON
      Flavor  Total Mentions  Unique Businesses  Cities  Avg Review Rating  Total Reviews
black sesame             344                102      29               4.38          96103
      matcha            5059               1245     172               4.29        1395670
         ube           10162               3688     346               3.79        4396811

PREPARING CENSUS DATA MATCHING
Found 102 unique black sesame businesses
✅ Saved black_sesame_locations_for_census.csv

✅ Analysis complete!
